### cabeçalho

#### importações

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

#### lendo as bases

In [18]:
URL_BASES = "./../Databases/test/"
patient_csv = "patient.csv"
infusiondrug = "infusiondrug.csv"
physicalExam = "physicalExam.csv"
lab = "lab.csv"

url_patient = URL_BASES + patient_csv
url_infusiondrug = URL_BASES + infusiondrug
url_physicalExam = URL_BASES + physicalExam
url_lab = URL_BASES + lab


#### pacientes septicos

In [3]:
df_patient = pd.read_csv(url_patient)
sepsis = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis', case=False, na=False)].reset_index()
sepsis = sepsis[['patientunitstayid']]
del df_patient
sepsis.head()

,patientunitstayid
0,151179
1,151900
2,156308
3,162841
4,166175


#### 24 horas lab

In [11]:
df_lab = pd.read_csv(url_lab)
df_lab_24 = df_lab.loc[(df_lab["labresultoffset"] >= 0) & (df_lab["labresultoffset"] <= 24*60)]
df_lab_24 = pd.merge(sepsis,df_lab_24,on='patientunitstayid')
del df_lab

#### 24 horas infusion drug

In [ ]:
df_infusiondrug = pd.read_csv(url_infusiondrug)
df_infusiondrug_24 = df_infusiondrug[(df_infusiondrug["infusionoffset"]>=0) &  (df_infusiondrug["infusionoffset"]<=60*24)]
df_infusiondrug_24 = pd.merge(sepsis,df_infusiondrug_24,on='patientunitstayid')
del df_infusiondrug

### tudo sobre physical exam

#### 24 horas physical exam

In [23]:
df_physicalExam = pd.read_csv(url_physicalExam)
df_physicalExam_24 = df_physicalExam[(df_physicalExam["physicalexamoffset"]>=0) &  (df_physicalExam["physicalexamoffset"]<=60*24)]
df_physicalExam_24 = pd.merge(sepsis,df_physicalExam_24,on='patientunitstayid')
del df_physicalExam

#### cada feature individual

In [24]:
def filter_exam(df, exam_name):
    return df[df['physicalexampath'].str.contains(exam_name, case=False, na=False)]

motor = filter_exam(df_physicalExam_24,'Motor Score')
eyes = filter_exam(df_physicalExam_24,'Eyes Score')
verbal = filter_exam(df_physicalExam_24,'Verbal Score')
systolic = filter_exam(df_physicalExam_24,'(systolic)')
systolic = filter_exam(systolic,'Current')
diastolic = filter_exam(df_physicalExam_24,'(diastolic)')
diastolic = filter_exam(diastolic,'Current')
ox_saturation = filter_exam(df_physicalExam_24,'O2 Sat%')
ox_saturation = filter_exam(ox_saturation,'Current')
heartRate = filter_exam(df_physicalExam_24,'HR Current')

del df_physicalExam_24

/tmp/ipykernel_141490/1620324637.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return df[df['physicalexampath'].str.contains(exam_name, case=False, na=False)]


#### editando as features

In [25]:
def edit_feature_exam(df,exam_name):
    df.drop(['physicalexamid', 'physicalexampath', 'physicalexamvalue'], axis=1, inplace=True)
    df.rename(columns={'physicalexamtext': exam_name}, inplace = True)

edit_feature_exam(motor,'motor')
edit_feature_exam(eyes,'eyes')
edit_feature_exam(verbal,'verbal')
edit_feature_exam(systolic,'systolic')
edit_feature_exam(diastolic,'diastolic')
edit_feature_exam(ox_saturation,'ox_saturation')
edit_feature_exam(heartRate,'heartRate')

In [26]:
systolic.head(1000)

,patientunitstayid,physicalexamoffset,systolic
5,151179,18,57
71,179269,108,93
120,193665,75,105
167,205928,79,90
200,219983,2,159
...,...,...,...
7354,3235765,2,104
7368,3327376,49,77
7395,3329305,19,93
7420,3341168,70,129


#### merge

In [27]:

df_physicalExam_24 = pd.merge(motor, eyes, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, verbal, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, systolic, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, diastolic, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, ox_saturation, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, heartRate, on=['patientunitstayid','physicalexamoffset'],how='outer')

del motor
del verbal
del eyes
del systolic
del diastolic
del ox_saturation
del heartRate

df_physicalExam_24.head()

,patientunitstayid,physicalexamoffset,motor,eyes,verbal,systolic,diastolic,ox_saturation,heartRate
0,151179,18,6,4,5,57,34,94,125
1,156308,39,6,4,5,NaN,NaN,100,96
2,162841,2,6,4,5,NaN,NaN,NaN,NaN
3,166175,37,6,4,5,NaN,NaN,97,120
4,179269,108,1,1,1,93,47,93,102


#### Mean arterial presure

In [17]:
# Tenta converter colunas para inteiro e usa pd.to_numeric com errors='coerce' em caso de falha
df_physicalExam_24["diastolic"] = pd.to_numeric(df_physicalExam_24["diastolic"], errors='coerce')
df_physicalExam_24["systolic"] = pd.to_numeric(df_physicalExam_24["systolic"], errors='coerce')

# Calcula o MAP (Mean Arterial Pressure) ignorando automaticamente valores nulos
df_physicalExam_24['MAP'] = (df_physicalExam_24['systolic'] + 2 * df_physicalExam_24['diastolic']) / 3


In [10]:
df_physicalExam_24.to_csv((URL_BASES+'physicalexam24.csv'),index=False)

### tudo sobre lab

#### cada feature individual

In [12]:
def filter_lab(df, lab_name):
    return df[df['labname'].str.contains(lab_name, case=False, na=False)]

alanine = filter_lab(df_lab_24, 'ALT')
aspartate = filter_lab(df_lab_24, 'AST')
albumin = filter_lab(df_lab_24, 'albumin')

bilirubin = filter_lab(df_lab_24, 'total bilirubin')
bands = filter_lab(df_lab_24, 'bands')
bicarbonate = filter_lab(df_lab_24, 'bicarbonate')
BUN = filter_lab(df_lab_24, 'BUN')

chloride = filter_lab(df_lab_24, 'chloride')
creatinine = filter_lab(df_lab_24, 'creatinine')
CRP = filter_lab(df_lab_24, 'CRP')

ESR = filter_lab(df_lab_24, 'ESR')

fio2 = filter_lab(df_lab_24, 'FiO2')

glucose = filter_lab(df_lab_24, 'glucose')

hemoglobin = filter_lab(df_lab_24, 'Hgb')

iqr = filter_lab(df_lab_24, 'INR')

lactate = filter_lab(df_lab_24, 'lactate')

pao2 = filter_lab(df_lab_24, 'paO2')
platelets = filter_lab(df_lab_24, 'platelets')

sodium = filter_lab(df_lab_24, 'sodium')

Temperature = filter_lab(df_lab_24, 'Temp')
troponin = filter_lab(df_lab_24, 'troponin')

WBC = filter_lab(df_lab_24, 'WBC x 1000')

del df_lab_24

#### editando features

In [13]:
def edit_feature_lab(df,lab_name):
    df.drop(['labid','labtypeid','labname','labresultrevisedoffset'],axis=1,inplace=True)
    df.rename(columns={'labresult': lab_name, 'labresulttext' : (lab_name+'resulttext'), 'labmeasurenamesystem' : (lab_name+'.unidMedida'), 'labmeasurenameinterface': (lab_name+'.measureInterface')},inplace=True)


edit_feature_lab(alanine,'alanine')
edit_feature_lab(aspartate,'aspartate')
edit_feature_lab(albumin,'albumin')

edit_feature_lab(bilirubin,'bilirubin')
edit_feature_lab(bicarbonate,'bicarbonate')
edit_feature_lab(bands,'bands')
edit_feature_lab(BUN,'bloodUreaNitrogen')

edit_feature_lab(creatinine,'creatinine')
edit_feature_lab(chloride,'chloride')
edit_feature_lab(CRP,'CRP')

edit_feature_lab(ESR,'ESR')

edit_feature_lab(fio2,'fio2')

edit_feature_lab(glucose,'glucose')

edit_feature_lab(hemoglobin,'hemoglobin')

edit_feature_lab(iqr,'iqr')

edit_feature_lab(lactate,'lactate')

edit_feature_lab(pao2,'pao2')
edit_feature_lab(platelets,'platelets')

edit_feature_lab(sodium,'sodium')

edit_feature_lab(troponin,'troponin')
edit_feature_lab(Temperature,'Temperature')
Temperature['celsius'] = 0
Temperature['Temperatureresulttext'] = Temperature['Temperatureresulttext'].astype(float)
Temperature['celsius'] = Temperature['Temperatureresulttext'].map(lambda x: x if x < 60 else (x - 32) * 5/9)

edit_feature_lab(WBC,'WBC')

#### merge

In [14]:
df_lab_24 = pd.merge(alanine,aspartate,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,albumin,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,BUN,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,bilirubin,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,bands,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,bicarbonate,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,CRP,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,chloride,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,creatinine,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,ESR,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,fio2,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,glucose,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,hemoglobin,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,iqr,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,lactate,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,pao2,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,platelets,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,sodium,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,Temperature,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,troponin,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,WBC,on=['patientunitstayid','labresultoffset'])
df_lab_24 = pd.merge(df_lab_24,WBCurine,on=['patientunitstayid','labresultoffset'])

del alanine
del aspartate
del albumin
del bilirubin
del bands
del bicarbonate
del BUN
del chloride
del creatinine
del CRP
del ESR
del fio2
del glucose
del hemoglobin
del iqr
del lactate
del pao2
del platelets
del sodium
del Temperature
del troponin
del WBC
del WBCurine


In [15]:
df_lab_24.head(10000)

,patientunitstayid,labresultoffset,alanine,alanineresulttext,alanine.unidMedida,alanine.measureInterface,aspartate,aspartateresulttext,aspartate.unidMedida,aspartate.measureInterface,...,troponin.unidMedida,troponin.measureInterface,WBC,WBCresulttext,WBC.unidMedida,WBC.measureInterface,WBCurine,WBCurineresulttext,WBCurine.unidMedida,WBCurine.measureInterface


### tudo sobre infusion drug

#### cada feature individual

In [ ]:
dopamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Norepinephrine', case=False, na=False)]

del df_infusiondrug_24

#### editando as features

In [ ]:

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()